In [1]:
!pip install paho-mqtt ipywidgets

   ---------------------------------------- 0.0/67.2 kB ? eta -:--:--
   ---------------------------------------- 67.2/67.2 kB 1.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import time
import json
import paho.mqtt.client as mqtt
import ipywidgets as widgets
from IPython.display import display

# ===== 설정 (AGV 코드와 똑같이) =====
BROKER_IP    = "10.236.94.221"  # 브로커 IP (아까 코드에 있던 그 IP)
BROKER_PORT  = 1883
TOPIC_DEV    = "robot/agv1/dev" # 치트키 토픽

# MQTT 연결
client = mqtt.Client(client_id=f"cheat-controller-{int(time.time())}")
try:
    client.connect(BROKER_IP, BROKER_PORT, keepalive=60)
    client.loop_start()
    print(f"✅ 리모컨 연결 성공! ({BROKER_IP})")
except Exception as e:
    print(f"❌ 연결 실패: {e}")

# ===== 버튼 클릭 함수 =====
def send_sync(area_name):
    payload = {
        "cmd": "fix_pose",
        "area": area_name
    }
    client.publish(TOPIC_DEV, json.dumps(payload), qos=0)
    print(f"🚀 [전송] 위치 강제 동기화 -> {area_name}")

# ===== UI 버튼 생성 =====
print("\n👇 로봇이 해당 위치를 지날 때 버튼을 누르세요 👇\n")

# 버튼 스타일 정의
layout = widgets.Layout(width='120px', height='40px', margin='5px')
btn_style = {'button_color': '#4CAF50', 'text_color': 'white'} # 초록색

# 장소 목록 (UserApp.txt 기준)
areas = ["CHARGE", "WATER", "DROP", "RES_A", "RES_B", "RES_C"]
btns = []

for area in areas:
    btn = widgets.Button(description=area, layout=layout)
    # 버튼 클릭 이벤트 연결
    # 주의: 람다식에서 변수 캡처 문제를 피하기 위해 default argument 사용
    btn.on_click(lambda b, a=area: send_sync(a))
    btns.append(btn)

# 버튼 가로로 배치 (3개씩)
row1 = widgets.HBox(btns[:3])
row2 = widgets.HBox(btns[3:])

display(row1, row2)

# 비상 정지 버튼 (옵션)
stop_btn = widgets.Button(description="🚨 긴급 정지", layout=widgets.Layout(width='380px', height='50px', margin='10px'))
stop_btn.style.button_color = '#FF5252' # 빨간색
def send_stop(b):
    client.publish("robot/agv1/task", json.dumps({"cmd": "stop"}), qos=0)
    print("🛑 [전송] 긴급 정지 명령")
stop_btn.on_click(send_stop)

display(stop_btn)

✅ 리모컨 연결 성공! (10.236.94.221)

👇 로봇이 해당 위치를 지날 때 버튼을 누르세요 👇



C:\Users\wp3wk\AppData\Local\Temp\ipykernel_27404\3953091696.py:13: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(client_id=f"cheat-controller-{int(time.time())}")


Button(description='🚨 긴급 정지', layout=Layout(height='50px', margin='10px', width='380px'), style=ButtonStyle(bu…